# Graph Construction

In [43]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly as py
#import plotly.graph_objects as go
from plotly.graph_objs import *
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
df = pd.read_csv('data_10000/data_2020-03-27_2020-03-28.csv')

In [97]:
df = df[:2000]

In [98]:
# Create retweet network from edgelist
G = nx.from_pandas_edgelist(
    df,
    source = 'username', 
    target = 'user_retweeted',
    create_using = nx.DiGraph())
    
# Print the number of nodes
print('Nodes in RT network:', len(G.nodes()))

# Print the number of edges
print('Edges in RT network:', len(G.edges()))

Nodes in RT network: 591
Edges in RT network: 497


In [99]:
# Some node-level features

centralScore = nx.betweenness_centrality(G)
inScore = G.in_degree()
outScore = G.out_degree()

In [100]:
max(list(dict(centralScore).values()))

3.453138039193117e-05

In [101]:
# Get a list of all nodeID in ascending order
nodeID = G.node.keys()
nodeID = sorted(nodeID)

In [102]:
# Node label information available on hover.
labels = []

for nd in nodeID:
      labels.append(nd + "<br>" + "In-retweets: " + str(inScore[nd]) + "<br>" + "Out-retweets: " + str(outScore[nd]) + "<br>" + "Centrality: " + str("%0.3f" % centralScore[nd]))

In [103]:
# Type of graph layout:
# spring, circular, bipatite, kamada_kawai, fruchterman_reingold, random, spectral, shell

pos=nx.spring_layout(G)

In [104]:
E=[tuple(e) for e in G.edges()]

In [105]:
N=len(labels)

In [106]:
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

width=800
height=800
layout=Layout(title= "Twitter Retweet Network with #confinement in Paris"+\
              "<br> Date : 26/03/2020",
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis={'showgrid': False, 'zeroline': False, 'visible': False},
    yaxis={'showgrid': False, 'zeroline': False, 'visible': False},
    #grid=None,
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text='This igraph.Graph has the Kamada-Kawai layout',
            xref='paper',
            yref='paper',
            x=0,
            y=-0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ]
    )

In [111]:
metric_list = list(dict(G.in_degree()).values()) #list(dict(G.out_degree()).values())
def get_metrics_array(metric_list) :
    metric_list = np.array(metric_list, dtype=np.float64)
    metric_list -= min(metric_list)
    metric_list /= max(metric_list)
    metric_list *= 35
    metric_list += 5
    return metric_list
metric =get_metrics_array(metric_list) 

In [112]:
Xv=[pos[k][0] for k in G.nodes()]
Yv=[pos[k][1] for k in G.nodes()]
Xed=[]
Yed=[]
for edge in E:
    Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
    Yed+=[pos[edge[0]][1],pos[edge[1]][1], None]

trace1=Scatter(x=Xed,
               y=Yed,
               mode='lines',
               line=dict(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )
trace2=Scatter(x=Xv,
               y=Yv,
               mode='markers',
               name='net',
               marker=dict(symbol='circle-dot',
                             size=metric,
                             color='#01B7EF',
                             line=dict(color='rgb(12,12,90)', width=0.5)
                             ),
               text=labels,
               hoverinfo='text'
               )

annot="Circular Layout"

data1=[trace1, trace2]
fig1=Figure(data=data1, layout=layout)
fig1['layout']['annotations'][0]['text']=annot

In [113]:
fig1.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
})

In [110]:
fig1.show()